In [1]:
import telegram, logging, soundfile, aifc, subprocess, os

from telegram.ext import Updater, CommandHandler, Filters, MessageHandler

from chatbotwrapper import PythonChatBot
from loggers import get_logger

In [2]:
chatbot = PythonChatBot(threshold=0.5)

In [3]:
train_log = get_logger('train_bot.log', logging.INFO)
test_log = get_logger('test_bot.log', logging.INFO)

In [4]:
bot_train = telegram.Bot(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

updater_train = Updater(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

dispatcher_train = updater_train.dispatcher

In [5]:
def start_train(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Я готов обучаться!\nПришли сообщением вопрос и ответ на него.\nНе забудь разделить их вот этим симоволом "|" """)

In [6]:
def train(bot, update):
    user = update.message.from_user
    text = update.message.text
    train_log.info("text of %s: %s" % (user.first_name, text))
    bot.send_message(chat_id=update.message.chat_id, text="""Спасибо, я постараюсь хорошо обучиться.\nПокидай обучалок еще, мне нравится учиться""")
    try:
        train_text = text.split('|')
        chatbot.train(train_text[0], train_text[1])
    except Exception as e:
        train_log.warning("Произошла ошибка -{}- \nНа обучающем тексте -{}-".format(e, text))
        bot.send_message(chat_id=update.message.chat_id, text="Не получилось обучиться. Может быть неправильный формат?")
    else:
        train_log.info("Обучение на обучающем тексте -{}- прошло успешно".format(text))

In [7]:
start_handler_train = CommandHandler('start', start_train)
text_handler_train = MessageHandler(Filters.text, train)

dispatcher_train.add_handler(start_handler_train)
dispatcher_train.add_handler(text_handler_train)

In [8]:
updater_train.start_polling()

In [9]:
bot_predict = telegram.Bot(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

updater_predict = Updater(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

dispatcher_predict = updater_predict.dispatcher

chatbot_predict = PythonChatBot(threshold=0.5, read_only=True)

In [10]:
def start_predict(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Меня еще учат, но кажется что я более-менее готов отвечать на вопросы""")

In [11]:
def predict(bot, update):
    user = update.message.from_user
    text = update.message.text
    test_log.info("text of %s: %s" % (user.first_name, text))
    try:
        response = chatbot_predict.response(text)
    except Exception as e:
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")
        test_log.warning("Произошла ошибка -{}- \nПытался ответить на -{}-".format(e, text))
    else:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        test_log.info("Ответ на вопрос -{}- прошел успешно".format(text))

In [12]:
def predict_sound(bot, update):
    import subprocess
    user = update.message.from_user
    filepath = '{}_question'.format(user.first_name)
    try:
        file_id = update.message.voice.file_id
        newFile = bot.get_file(file_id)
        newFile.download(filepath + '_wb.mp3')
        subprocess.call(['ffmpeg', '-i', filepath + '_wb.mp3', filepath + '_wb.wav'])
        answer = chatbot_predict.response_sound(filepath + '_wb.wav')
        os.remove(filepath+'_wb.mp3')
        os.remove(filepath+'_wb.wav')
        bot.send_message(chat_id=update.message.chat_id, text=str(answer))
        test_log.info("Ответ на вопрос -{}- прошел успешно. Ответ на звук".format(str(answer)))
    except Exception as e:
        test_log.warning("Произошла ошибка -{}- \nПытался ответить на звук".format(e))
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")

In [13]:
start_handler_predict = CommandHandler('start', start_predict)
text_handler_predict = MessageHandler(Filters.text & (~Filters.voice), predict)
voice_handler_predict = MessageHandler(Filters.voice, predict_sound)

dispatcher_predict.add_handler(start_handler_predict)
dispatcher_predict.add_handler(text_handler_predict)
dispatcher_predict.add_handler(voice_handler_predict)

In [14]:
updater_predict.start_polling()

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
